In [1]:
# This is a Python 3 environment

# Base level imports for data science work
import numpy as np 
import pandas as pd
import re,string,unicodedata
import os
from os import path
from json import decoder
import glob, os, json
import re

# Visualization Libs
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# NLP Libs
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
from keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# Additional Libs
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet

# Deep Learning Libs
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [2]:
#Herunterladen des Data Frame
df = pd.read_csv("https://chruezfalsch.ch/df_BA_cleaned_400-3500.csv")
del df['Unnamed: 0'] 

In [3]:
X = df.text
y = df.fake

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8,stratify=y, random_state=19)

# Create of Count Vectorize object
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
#cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
cv=CountVectorizer(min_df=0.0,max_df=1.0,binary=False,ngram_range=(1,3))
cv=cv.fit(X_train)
# Transform the train dataset
cv_train=cv.fit_transform(X_train)

# Transform the test dataset
cv_test=cv.transform(X_test)

print('Bag of words Train:',cv_train.shape)
print('Bag of words Test:',cv_test.shape)

Bag of words Train: (1306, 1137107)
Bag of words Test: (327, 1137107)


In [4]:
# Create the TF-IDF object
#tfidf=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
tfidf=TfidfVectorizer(min_df=0.0,max_df=1.0,use_idf=True,ngram_range=(1,3))
tfidf=tfidf.fit(X_train)

# Transform the train dataset
tfidf_train=tfidf.fit_transform(X_train)

# Transform the test dataset
tfidf_test=tfidf.transform(X_test)


print('TF-IDF Train:',tfidf_train.shape)
print('TF-IDF Test:',tfidf_test.shape)

TF-IDF Train: (1306, 1137107)
TF-IDF Test: (327, 1137107)


In [5]:
# Create our model object
mnb = MultinomialNB()

# Fit the model to the Bag of words CountVectorizor
mnb_cv = mnb.fit(cv_train, y_train)


# Fit the model to the TF-IDF features
mnb_tfidf = mnb.fit(tfidf_train, y_train)

In [6]:
# Predicting the model for bag of words
#mnb_cv_predict=mnb.predict(cv_test)
mnb_cv_predict=mnb_cv.predict(cv_test)

# Predicting the model for tf-idf features
#mnb_tfidf_predict=mnb.predict(tfidf_test)
mnb_tfidf_predict=mnb_tfidf.predict(tfidf_test)

In [7]:
# Check the accuracy score for bag of words
mnb_cv_score=accuracy_score(y_test, mnb_cv_predict)
print("Naive Bayes Bag of words accuracy score:", mnb_cv_score)

# Check the accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(y_test,mnb_tfidf_predict)
print("Naive Bayes TF-IDF accuracy score:", mnb_tfidf_score)

Naive Bayes Bag of words accuracy score: 0.9235474006116208
Naive Bayes TF-IDF accuracy score: 0.9357798165137615


In [8]:
mnb_cv_report = classification_report(y_test, mnb_cv_predict, target_names = ['0','1'])
print(mnb_cv_report)
mnb_tfidf_report = classification_report(y_test, mnb_tfidf_predict, target_names = ['0','1'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

           0       0.87      0.99      0.93       159
           1       0.99      0.86      0.92       168

    accuracy                           0.92       327
   macro avg       0.93      0.93      0.92       327
weighted avg       0.93      0.92      0.92       327

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       159
           1       0.97      0.90      0.94       168

    accuracy                           0.94       327
   macro avg       0.94      0.94      0.94       327
weighted avg       0.94      0.94      0.94       327



#Export

In [9]:
import pickle
from joblib import dump, load

In [10]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
#dump(mnb, 'ba_model_1.joblib')
#!cp ba_model_1.joblib "/content/drive/MyDrive/BA/Project/ba_model_1.joblib"

#dump(mnb_cv, 'ba_model_1_cv.joblib')
#!cp ba_model_1_cv.joblib "/content/drive/MyDrive/BA/Project/ba_model_1_cv.joblib"

#dump(mnb_tfidf, 'ba_model_1_tfidf.joblib')
#!cp ba_model_1_tfidf.joblib "/content/drive/MyDrive/BA/Project/ba_model_1_tfidf.joblib"

In [12]:
pickle.dump(mnb, open("/content/drive/MyDrive/BA/Project/ba_model_3.joblib", 'wb'))
pickle.dump(cv, open("/content/drive/MyDrive/BA/Project/ba_model_3_cv.joblib", 'wb'))
pickle.dump(tfidf, open("/content/drive/MyDrive/BA/Project/ba_model_3_tfidf.joblib", 'wb'))